# <center>Big Data &ndash; Exercise 1</center>
## <center>Fall 2024 &ndash; Week 1 &ndash; ETH Zurich</center>

### Aims
- **After this exercise:** 
    - Understand the SQL language and its common query patterns.
    - Understand the 'table' data shape, normalization, and when they can (and should) be used.
    - be able to query data in tables with the SQL language.
- **Later in the semester:** 
    - Relate these language features and query patterns relative to other data shapes, technologies, and the languages designed to query them.
    - Understand when tables are not the appropriate shape for your data and when you can (and should) throw normalization away!

### Prerequisites
In this exercise, you will brush-up the fundamental concepts of relational databases and SQL. If you haven't taken an introductory databases course (or want to refresh your knowledge) we recommend you to read the following:

Garcia-Molina, Ullman, Widom: Database Systems: The Complete Book. Pearson, 2. Edition, 2008. (Chapters 1, 2, 3, and 6) [Available in the ETH Library] [[Online]](https://ebookcentral.proquest.com/lib/ethz/detail.action?pq-origsite=primo&docID=5832965) [[Selected solutions]](http://infolab.stanford.edu/~ullman/dscbsols/sols.html).

Or have a look at the recordings from Information Systems for Engineers - ETH Zurich, available on [[YouTube]](https://www.youtube.com/c/GhislainFournysLectures).

### Database Set-up
We will be once again working in the ExamMagicBox (you can find it in the following [[link]](https://polybox.ethz.ch/index.php/s/wa57XqDKkxRMb0q) if you have not downloaded it yet): please drag this Notebook in the folder. Just like last week, activate the docker container for the exercise sheet with `docker compose up`; please wait for the message `PostgreSQL init process complete; ready for start up` in the docker logs before proceeding! Alternatively you can start the Docker with `docker compose up -d` and wait for the command to execute: please note that you are creating the containers in the background this way. You can then type `docker compose down` when you are done.

As before, we set up our connection to the database and enable use of `%sql` and `%%sql`.

In [2]:
server='db'
user='postgres'
password='example'
database='postgres'
connection_string=f'postgresql://{user}:{password}@{server}:5432/{database}'

In [3]:
%reload_ext sql
%sql $connection_string

In [4]:
%%sql
SELECT version();

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


version
"PostgreSQL 16.2 (Debian 16.2-1.pgdg120+2) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit"


### Origin of the data
You can find more information on the dataset in the following links
- [Discogs](https://www.discogs.com/)
- [Discogs XML data dumps](http://data.discogs.com/)

If you do not want to use Docker or it does not work you can download the dataset from this [link](https://cloud.inf.ethz.ch/s/DtjCHTLRHT39BRN/download/discogs.dump.xz), see `postgres-init.sh` to see how to import it)

## Exercise 1: Explore the dataset
We want to first understand the dataset a bit better. You will find some queries below to help you explore the schema.

### List tables
The following query retrieves a list of tables in the database from a system table describing the current database.

In [5]:
%%sql 
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public';

 * postgresql://postgres:***@db:5432/postgres
22 rows affected.


table_name
companies
jobs
employees02
badges
comments
inventory
posthistory
postlinks
posts
tags


### List attributes/columns
The following query retrieves a list of columns from the tables in the database.

In [6]:
%%sql 
SELECT table_name, column_name, data_type, is_nullable, ordinal_position
FROM information_schema.columns
WHERE table_schema = 'public' AND table_name IN ('artists', 'released_by', 'releases', 'tracks')
AND table_name NOT LIKE 'pg_%'
ORDER BY table_name, ordinal_position;

 * postgresql://postgres:***@db:5432/postgres
17 rows affected.


table_name,column_name,data_type,is_nullable,ordinal_position
artists,artist_id,integer,NO,1
artists,name,character varying,YES,2
artists,realname,text,YES,3
artists,profile,text,YES,4
artists,url,text,YES,5
released_by,release_id,integer,NO,1
released_by,artist_id,integer,NO,2
releases,release_id,integer,NO,1
releases,released,date,NO,2
releases,title,text,NO,3


### Have a look at the datasets
The following simple query gives the first 5 rows of the `artists` dataset

In [7]:
%%sql
SELECT * FROM artists LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
1,The Persuader,Jesper Dahlbäck,None,None
2,Mr. James Barth & A.D.,Cari Lekebusch & Alexi Delano,None,None
3,Josh Wink,Joshua Winkelman,"After forming [l=Ovum Recordings] as an independent label in October 1994 with former partner [a=King Britt], Josh recorded the cult classic 'Liquid Summer'. He went on to release singles for a wide variety of revered European labels ranging from Belgium's [l=R & S Records] to England's [l=XL Recordings]. In 1995, Wink became one of the first DJ-producers to translate his hard work into mainstream success when he unleashed a string of classics including 'Don't Laugh'¸ 'I'm Ready' and 'Higher State of Consciousness' that topped charts worldwide. More recently he has had massive club hits such as 'How's Your Evening So Far' and 'Superfreak' but he has also gained a lot of attention trough his remixes for [a=FC Kahuna], [a=Paul Oakenfold], [a=Ladytron], [a=Clint Mansell], [a=Sting] and [a=Depeche Mode], among others.",http://www.joshwink.com/
4,Johannes Heil,Johannes Heil,"Electronic music producer, musician and live performer, born 3 February 1978 near the town of Bad Nauheim, Germany. Founder of [l=JH] and [l=Metatron Recordings].",http://johannes-heil.com/
5,Heiko Laux,Heiko Laux,German DJ and producer based in Berlin. He is the founder of [l=Kanzleramt].,http://www.heiko-laux.com


Naturally we could write similar queries to better understand each of the other tables.

#### With what you now know about the datasets, try to answer the following questions

1. Which concepts are modelled in the dataset and how do they relate to each other? <b>Hint</b>: how do the tables connect logically?
2. Why do you think this shape (table) was chosen for the data and why not the other shapes?
3. In which normal forms are the corresponding relations?
4. How can we denormalise the data to make some queries more efficient? <b>Hint</b>: have a look at the queries in the next session of the exercises to see if adding some columns to some tables could reduce the need to `JOIN`.
5. What potential problems could result from adding redundancy?

## Exercise 2: SQL warm-up
Now that we familiarised ourself with the tables and relationship, we will begin with several SQL queries to ease us back into the language.

<b>Practical tips:</b>
- You might want to begin by retrieving a few rows from each of the database tables to get a sense of what is stored. 
- When testing your queries, it is good practice to add a "LIMIT" clause to avoid inadvertedly retrieving hundreds of rows.

The following is an example query that contains some common SQL expressions. A complete list can be found at: https://www.postgresql.org/docs/current/sql-select.html

In [8]:
%%sql
SELECT DISTINCT
    a.name AS column1,
    COUNT(t.track_id) AS column2,
    AVG(t.duration) AS column3
FROM
    artists a
    JOIN released_by rb USING(artist_id)
    JOIN releases r USING(release_id)
    JOIN tracks t USING(release_id)
WHERE
    t.duration > 123
    AND t.title != 'My Query'
    AND r.country = 'Switzerland'
GROUP BY
    a.artist_id, a.name
HAVING
    COUNT(t.track_id) > 0
ORDER BY
    column2 DESC,
    column3 DESC
LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


column1,column2,column3
Various Artists,2814,349.5806680881307747
DJ Snowman,548,287.5620437956204380
DJ Noise,504,401.8650793650793651
Dave 202,425,326.8800000000000000
DJ Nonsdrome,242,306.3842975206611570


The following is a visual representation of the database schema for quick reference.

<img src="https://polybox.ethz.ch/index.php/s/8CqNffQrR0EDbuC/download" width=800/>

#### 1. Retrieve all releases that were released after January 1, 2017.

In [9]:
%%sql
SELECT r.title, r.released
FROM releases AS r
WHERE r.released > '2017-01-01'

 * postgresql://postgres:***@db:5432/postgres
18 rows affected.


title,released
"Kizomba Mix, Vol. 2 [2018] 2 CDs",2018-12-14
Shed01,2018-01-01
Jetlag Disco,2018-01-01
The Bad Behaviour E.P,2018-01-01
Volume 1,2018-01-01
Sudd. Autumn Collection 03,2018-01-01
Technikal Support,2018-01-01
You Wanna Do What / In One Hand,2018-01-01
Happy To Be Sad / I Was Just Leaving,2018-01-01
No-Harm,2018-01-01


#### 2. Find all tracks with a duration longer than 7 hours. Assume the 'duration' column in the 'tracks' table is in seconds.

In [10]:
%%sql
SELECT *
FROM tracks t
WHERE t.duration > 7 * 60 * 60

 * postgresql://postgres:***@db:5432/postgres
3 rows affected.


release_id,position,title,duration,track_id
47796,9,Rapper's Relight,25579,256970
47796,11,Dialectical Transformation III Peace In Rwanda Mix,27196,256972
478281,4,Live 1996.12.30.,31934,2526159


#### 3. Retrieve the titles of 5 releases along with the names of the artists who released them.

In [11]:
%%sql
SELECT
    r.title,
    a.name
FROM releases AS r
JOIN released_by AS rb ON r.release_id = rb.release_id
JOIN artists AS a ON rb.artist_id = a.artist_id
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


title,name
Stockholm,The Persuader
Knockin' Boots Vol 2 Of 2,Mr. James Barth & A.D.
Profound Sounds Vol. 1,Josh Wink
Flowerhead,DATacide
Knockin' Boots (Vol 1 Of 2),Mr. James Barth & A.D.


#### 4. List each genre and the number of releases in that genre.

In [12]:
%%sql
SELECT
    r.genre,
    COUNT(r.*) AS num_releases
FROM releases AS r
GROUP BY r.genre

 * postgresql://postgres:***@db:5432/postgres
15 rows affected.


genre,num_releases
Blues,48
Brass & Military,4
Children's,6
Classical,257
Electronic,183766
"Folk, World, & Country",101
Funk / Soul,3674
Hip Hop,10598
Jazz,3325
Latin,153


#### 5. Identify the top 5 artists who have the most releases.

In [13]:
%%sql
SELECT
    a.artist_id,
    a.name,
    COUNT(r.*) AS num_releases
FROM artists AS a
JOIN released_by AS rb ON rb.artist_id = a.artist_id
JOIN releases AS r ON r.release_id = rb.release_id
GROUP BY a.artist_id
ORDER BY num_releases DESC
LIMIT 5


 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,num_releases
194,Various Artists,46123
8760,Madonna,617
7552,Pet Shop Boys,600
4118,Faithless,336
15885,Michael Jackson,332


#### 6. Find the artist who has the longest total duration of tracks across all their releases.

In [14]:
%%sql
SELECT
    a.artist_id,
    a.name,
    SUM(t.duration) AS total_duration
FROM artists AS a
JOIN released_by AS rb ON rb.artist_id = a.artist_id 
JOIN tracks AS t ON t.release_id = rb.release_id
GROUP BY a.artist_id
ORDER BY total_duration DESC
LIMIT 1

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,total_duration
194,Various Artists,227180023


#### 7. Find how many releases that have tracks with duplicate titles.

In [15]:
# %%sql
# /*
# Why so slow?
# SELECT COUNT(DISTINCT r.release_id) AS num_releases_with_dup
# FROM releases AS r
# WHERE EXISTS(
#     SELECT t.title, COUNT(DISTINCT t.release_id) AS title_count
#     FROM tracks AS t
#     WHERE t.release_id = r.release_id
#     GROUP BY t.title
#     HAVING COUNT(title) > 1
# )
# */


In [16]:

%%sql
SELECT r.release_id, t.title, COUNT(t.title) AS tile_occurs
FROM releases AS r
JOIN tracks AS t ON r.release_id = t.release_id
WHERE t.release_id = r.release_id
GROUP BY r.release_id, t.title
HAVING COUNT(t.title) > 1
LIMIT 10


 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


release_id,title,tile_occurs
110,Untitled,3
466,Scarlet,2
695,Communications From The Lab,2
695,Last Tribute From The 20th Century,2
710,Meanwhile Back At Home,2
710,Untitled,2
734,Der Klang Der Familie (F.U.S.E. Mix),2
770,Untitled,4
1034,Untitled,3
1140,Always (Original Mix),2


In [17]:
%%sql
WITH releases_with_dup AS (
    SELECT r.release_id, t.title, COUNT(t.title) AS tile_occurs
    FROM releases AS r
    JOIN tracks AS t ON r.release_id = t.release_id
    WHERE t.release_id = r.release_id
    GROUP BY r.release_id, t.title
    HAVING COUNT(t.title) > 1
)

SELECT COUNT(DISTINCT rwd.release_id) AS num_releases_with_dup
FROM releases_with_dup AS rwd

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


num_releases_with_dup
9046


#### 8. Retrieve the artists with the name of 'Coldplay'.

In [18]:
%%sql
SELECT a.*
FROM artists AS a
WHERE a.name = 'Coldplay'

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


artist_id,name,realname,profile,url
29735,Coldplay,None,"Coldplay is an English rock band from London, England. They've been a band since January 16, 1998 when they lost a demotape competition on XFM in London. Philip Christopher Harvey is the band's manager. [b][u]Line-up:[/u][/b] Jonny Buckland (Jonathan Mark Buckland) - Guitar Will Champion (William Champion) - Drums Guy Berryman (Guy Rupert Berryman) - Bass Chris Martin (Christopher Anthony John Martin) - Vocals",http://coldplay.com/


#### 9. List the titles of all releases by that artist in alphabetical order.
<b>Hint</b>: Ignore the fact that different relases can have the same title.

In [19]:
%%sql
SELECT DISTINCT r.title
FROM releases AS r
JOIN released_by AS rb ON rb.release_id = r.release_id
JOIN artists AS a ON a.artist_id = rb.artist_id
WHERE a.name = 'Coldplay'

 * postgresql://postgres:***@db:5432/postgres
40 rows affected.


title
Acoustic
A Rush Of Blood To The Head
Boot Of Sound
Brothers & Sisters
Clocks
Clocks...
Clocks / Chime Trance Remixes
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Planet Rockers Remixes)


#### 10. How many tracks from 'Coldplay' have position '1'?

In [20]:
%%sql
SELECT COUNT(t.*) AS amount
FROM tracks AS t
JOIN released_by AS rb ON rb.release_id = t.release_id
JOIN artists AS a ON a.artist_id = rb.artist_id
WHERE a.name = 'Coldplay'
AND t.position = '1'

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


amount
32


#### 11. List the titles of all releases by Coldplay that contain less than 2 tracks.

In [21]:
%%sql
SELECT DISTINCT r.title
FROM releases AS r
JOIN released_by AS rb ON rb.release_id = r.release_id
JOIN artists AS a ON a.artist_id = rb.artist_id
JOIN tracks AS t ON t.release_id = r.release_id
WHERE a.name = 'Coldplay'
GROUP BY r.release_id, r.title
HAVING COUNT(t.*) < 2

 * postgresql://postgres:***@db:5432/postgres
14 rows affected.


title
Boot Of Sound
Clocks
Clocks (Cosmos Rmx)
Clocks (Dean Coleman Remix)
Clocks (Remix)
God
In My Place
One I Love
Speed Of Sound (Karl G Remix)
Talk


#### 12. What is the average track duration?

In [22]:
%%sql
SELECT AVG(t.duration) AS avg_duration
FROM tracks AS t

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg_duration
325.0749298696787647


#### 13. How many artists have released tracks longer than twice the average?

In [23]:
%%sql
SELECT COUNT(a.*)
FROM artists AS a
WHERE EXISTS
(
    SELECT *
    FROM tracks t
    JOIN released_by AS rb ON t.release_id = rb.release_id
    WHERE rb.artist_id = a.artist_id
    AND t.duration > 2 * (SELECT AVG(t.duration) AS avg_duration FROM tracks AS t)
)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
6386


## Exercise 3: more SQL
We will now see more complex SQL queries.

<b>Practical tips:</b>

When writing complex queries, you might want to split them into smaller parts by using <b>Common Table Expressions</b> (CTEs). A CTE is a named temporary result set that you can reference within statements (SELECT, INSERT, UPDATE, ... ). You can find more about CTEs at: https://www.postgresql.org/docs/current/queries-with.html

The following is an example of a query using two CTEs:

In [24]:
%%sql
WITH countries AS (
    SELECT DISTINCT country FROM releases
),
genres AS (
    SELECT DISTINCT genre FROM releases
)
SELECT c.country as column1, g.genre as column2
FROM countries c, genres g
LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


column1,column2
Argentina,Blues
Argentina,Brass & Military
Argentina,Children's
Argentina,Classical
Argentina,Electronic


In some exercises, you might also want to use <b>subqueries</b>. A subquery is a nested query, usually with the purpose of retrieving data that will be used in in the outer query. For instance, subqueries can appear in WHERE, FROM and SELECT clauses.

The following is an example of a query than includes a subquery:

In [25]:
%%sql
SELECT release_id as column1 FROM (
    SELECT release_id, title, COUNT(*) FROM tracks
    GROUP BY release_id, title
    HAVING COUNT(*) > 1
) sub
LIMIT 5;

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


column1
110
466
695
695
710


#### 1. What is the title of the album from 'Coldplay' with the most amount of tracks?

In [96]:
%%sql
SELECT r.release_id, r.title, COUNT(t.track_id) AS num_tracks
FROM releases AS r
JOIN tracks AS t ON r.release_id = t.release_id
JOIN released_by AS rb ON r.release_id = rb.release_id
JOIN artists AS a ON a.artist_id = rb.artist_id
WHERE a.name = 'ColdPlay'
GROUP BY r.release_id, r.title
ORDER BY num_tracks DESC
LIMIT 10

 * postgresql://postgres:***@db:5432/postgres
0 rows affected.


release_id,title,num_tracks


In [99]:
%%sql
SELECT r.release_id, r.title, COUNT(t.track_id) AS num_tracks
FROM tracks AS t
JOIN releases AS r USING(release_id)
JOIN released_by AS rb USING(release_id)
JOIN artists AS a USING(artist_id)
WHERE a.name = 'Coldplay'
GROUP BY r.release_id
ORDER BY num_tracks DESC
LIMIT 1

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


release_id,title,num_tracks
617736,X&Y (Special Dutch Edition),19


#### 2. What is the name of the first artist in alphabetical order with releases in the most genres. Please make sure to exclude "Various Artists".

In [102]:
%%sql
SELECT a.artist_id, a.name, COUNT(DISTINCT r.genre) AS num_genres
FROM artists AS a
JOIN released_by AS rb USING(artist_id)
JOIN releases AS r USING(release_id)
WHERE a.name <> 'Various Artists'
GROUP BY a.artist_id, a.name
ORDER BY num_genres DESC, a.name ASC
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,num_genres
47742,Diana Ross,7
15885,Michael Jackson,7
18956,Stevie Wonder,7
262940,The London Symphony Orchestra,7
15900,Ennio Morricone,6


#### 3. In what year did they (the artist from the previous question) release their first album?

In [107]:
%%sql
WITH
arists_by_num_genres AS (
    SELECT a.artist_id, a.name, COUNT(DISTINCT r.genre) AS num_genres
    FROM artists AS a
    JOIN released_by AS rb USING(artist_id)
    JOIN releases AS r USING(release_id)
    WHERE a.name <> 'Various Artists'
    GROUP BY a.artist_id, a.name
    ORDER BY num_genres DESC, a.name ASC
),
artist_with_most_genres AS (
    SELECT abng.artist_id, abng.name
    FROM arists_by_num_genres AS abng
    LIMIT 1
)

SELECT
    r.release_id, r.title, r.released
FROM releases AS r
JOIN released_by AS rb USING (release_id)
WHERE rb.artist_id IN (SELECT awmg.artist_id FROM artist_with_most_genres AS awmg)
GROUP BY r.release_id
ORDER BY r.released ASC
LIMIT 1

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


release_id,title,released
391739,Greatest Hits,1967-01-01


#### 4. How many artists have released an album with total track duration above twice the average total track duration?

<b>Hint</b>: this is not the same as exercise 2.13 since we are lookong at the <b>total</b> track duration of the album.

In [48]:
%%sql
SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
FROM releases AS r
JOIN tracks AS t USING(release_id)
GROUP BY r.release_id
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


release_id,title,total_track_duration
1,Stockholm,1771
2,Knockin' Boots Vol 2 Of 2,1117
3,Profound Sounds Vol. 1,4053
5,Flowerhead,3879
6,Knockin' Boots (Vol 1 Of 2),934


In [49]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
)

SELECT rbd.total_track_duration
FROM releases_by_duration AS rbd
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


total_track_duration
1771
1117
4053
3879
934


In [51]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
),

total_track_durations AS (
    SELECT rbd.total_track_duration
    FROM releases_by_duration AS rbd
)

SELECT AVG(ttd.total_track_duration)
FROM total_track_durations AS ttd

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


avg
2496.3457361965954456


In [56]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
),

total_track_durations AS (
    SELECT rbd.total_track_duration
    FROM releases_by_duration AS rbd
)

SELECT rbd.*
FROM releases_by_duration rbd
WHERE rbd.total_track_duration > 2 * (SELECT AVG(ttd.total_track_duration) FROM total_track_durations AS ttd)
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


release_id,title,total_track_duration
320,Avantgardism Vol. 2 (One Giant Bleep For Mankind),9272
333,Lowercase,8364
334,Further East-Westercisms,8934
335,Minimalism: More Or Less,9162
336,East - Westercisms Vol. 3,8732


In [59]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
),

total_track_durations AS (
    SELECT rbd.total_track_duration
    FROM releases_by_duration AS rbd
),

above_avg_releases_by_duration AS (
    SELECT rbd.*
    FROM releases_by_duration rbd
    WHERE rbd.total_track_duration > 2 * (SELECT AVG(ttd.total_track_duration) FROM total_track_durations AS ttd)
)

SELECT
    COUNT(DISTINCT a.artist_id)
FROM artists AS a
JOIN released_by AS rb USING (artist_id)
WHERE rb.release_id IN (SELECT aarbd.release_id FROM above_avg_releases_by_duration AS aarbd)

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


count
3388


#### 5. How many artists have both a release with a track longer than twice the average and one with total duration longer than twice the average?

<b>Hint</b>: you can use `INTERSECT` or `EXISTS` to write your query.

In [62]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
),

total_track_durations AS (
    SELECT rbd.total_track_duration
    FROM releases_by_duration AS rbd
),

above_avg_releases_by_duration AS (
    SELECT rbd.*
    FROM releases_by_duration rbd
    WHERE rbd.total_track_duration > 2 * (SELECT AVG(ttd.total_track_duration) FROM total_track_durations AS ttd)
),

artists_with_above_avg_releases AS (
    SELECT a.*
    FROM artists AS a
    JOIN released_by AS rb USING (artist_id)
    WHERE rb.release_id IN (SELECT aarbd.release_id FROM above_avg_releases_by_duration AS aarbd)
),

artists_with_above_avg_tracks AS (
    SELECT a.*
    FROM artists AS a
    WHERE EXISTS
    (
        SELECT *
        FROM tracks t
        JOIN released_by AS rb ON t.release_id = rb.release_id
        WHERE rb.artist_id = a.artist_id
        AND t.duration > 2 * (SELECT AVG(t.duration) AS avg_duration FROM tracks AS t)
    )
)

SELECT ar.artist_id
FROM artists_with_above_avg_releases AS ar

INTERSECT

SELECT at.artist_id
FROM artists_with_above_avg_tracks AS at

LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id
56202
18803
259644
38885
1552


In [64]:
%%sql
WITH
releases_by_duration AS (
    SELECT r.release_id, r.title, SUM(t.duration) AS total_track_duration
    FROM releases AS r
    JOIN tracks AS t USING(release_id)
    GROUP BY r.release_id
),

total_track_durations AS (
    SELECT rbd.total_track_duration
    FROM releases_by_duration AS rbd
),

above_avg_releases_by_duration AS (
    SELECT rbd.*
    FROM releases_by_duration rbd
    WHERE rbd.total_track_duration > 2 * (SELECT AVG(ttd.total_track_duration) FROM total_track_durations AS ttd)
),

artists_with_above_avg_releases AS (
    SELECT a.*
    FROM artists AS a
    JOIN released_by AS rb USING (artist_id)
    WHERE rb.release_id IN (SELECT aarbd.release_id FROM above_avg_releases_by_duration AS aarbd)
),

artists_with_above_avg_tracks AS (
    SELECT a.*
    FROM artists AS a
    WHERE EXISTS
    (
        SELECT *
        FROM tracks t
        JOIN released_by AS rb ON t.release_id = rb.release_id
        WHERE rb.artist_id = a.artist_id
        AND t.duration > 2 * (SELECT AVG(t.duration) AS avg_duration FROM tracks AS t)
    )
),

intesect_artists AS (
    SELECT ar.artist_id
    FROM artists_with_above_avg_releases AS ar

    INTERSECT

    SELECT at.artist_id
    FROM artists_with_above_avg_tracks AS at
)

SELECT COUNT(DISTINCT ia.artist_id) AS num_artists
FROM intesect_artists AS ia

 * postgresql://postgres:***@db:5432/postgres
1 rows affected.


num_artists
1870


#### 6. Show the artists have more than 200 releases in total but have no releases with the genre 'Pop' in reversed alphabetical order.

In [80]:
%%sql
SELECT a.artist_id, a.name, COUNT(DISTINCT r.release_id) AS num_releases
FROM artists AS a
JOIN released_by AS rb USING(artist_id)
JOIN releases AS r USING(release_id)
GROUP BY a.artist_id
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,num_releases
1,The Persuader,10
2,Mr. James Barth & A.D.,3
3,Josh Wink,159
4,Johannes Heil,57
5,Heiko Laux,45


In [84]:
%%sql
WITH artists_by_num_releases AS (
    SELECT a.*, COUNT(DISTINCT r.release_id) AS num_releases
    FROM artists AS a
    JOIN released_by AS rb USING(artist_id)
    JOIN releases AS r USING(release_id)
    GROUP BY a.artist_id
)

SELECT a.*
FROM artists AS a
JOIN artists_by_num_releases AS abnr USING(artist_id)
WHERE abnr.num_releases > 200
LIMIT 5


 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
194,Various Artists,None,Placeholder artist for Various,None
1279,Orbital,Paul & Phil Hartnoll,"Techno outfit, founded in the late 80s by the brothers Paul and Phil Hartnoll.",http://orbitalofficial.com
1280,Underworld,"Karl Hyde, Rick Smith, (Darren Emerson)","UK electronic music act formed by Rick Smith and [a=Karl Hyde] in 1987 after their former band [a=Freur] disbanded. The first line-up released more song oriented funky guitar based pop over the course of two albums. In 1991 the core duo returned with a new track oriented technoid direction and a new ally; DJ [a=Darren Emerson]. In 1996 the former b-side ""Born Slippy NUXX"" was used in Danny Boyle's film ""Trainspotting"" and became an instant anthem of the mid to late 1990's. Emerson left in 2000 after the release of ""Everything, Everything"" to pursue a solo career and was replaced on tour DJ duties by [a=Darren Price]. Price became a performing member of Underworld Live in 2005.",http://www.underworldlive.com
4118,Faithless,None,"Faithless officially split up in April 2011. Some of the original members (Sister Bliss, Maxi Jazz and percussionist Sudha Kheterpal) continue to perform under the name Faithless Sound System. Faithless were [a=Maxwell Frazer] AKA [a=Maxi Jazz] (Conscious Rap) [a=Ayalah Bentovim] AKA [a=Sister Bliss] (Music) Roland Armstrong AKA [a=Rollo] (Producer, Music). Many of the songs are sung by [a=Pauline Taylor]. [a=Jamie Catto] was originally part of the group but left after the release of the second album ""Sunday 8PM"". [a=Zoë Johnston] joined for the album ""Outrospective"". Their first album, 1996's ""Reverence"", was an underground sensation but did not quite break the mainstream. 1998's ""Sunday 8AM"" changed all of that, the single ""God Is a DJ"" became a big club anthem while the album enjoyed bigger sales than anything they had previously released. Critics may have been harsh, but the group kept moving forward with all members also pursuing solo careers on the side. When playing live they were joined by guitarist [a=Dave Randall (2)] and bassist, [a=Aubrey Nunn].",http://www.faithless.co.uk
5797,ATB,André Tanneberger,"German Producer ATB - [b]A[/b]ndré [b]T[/b]anne[b]b[/b]erger Born on February 26, 1973 in Freiberg, Saxony, Germany.",http://www.atb-music.com/


In [85]:
%%sql
SELECT a.*
FROM artists AS a
JOIN released_by AS rb USING(artist_id)
JOIN releases AS r USING(release_id)
WHERE r.genre = 'Pop'
LIMIT 5

 * postgresql://postgres:***@db:5432/postgres
5 rows affected.


artist_id,name,realname,profile,url
2807,Saint Etienne,"Sarah Cracknell, Bob Stanley & Pete Wiggs.","English alternative pop band, founded in London in 1990. They are named after the French football team AS Saint-Étienne.",http://www.saintetienne.com
235979,Boney M.,None,"Producer [a=Frank Farian] created Boney M. as a studio project in 1975. On the first output, ""Baby Do You Wanna Bump"", Farian performed all vocals himself. Soon, however, TV stations started to demand 'Boney M.' to perform the track on TV shows, which made Farian form an actual group. After some initial changes, Frank Farian arrived at the lineup with which the group had their biggest successes: Maizie Williams, Marcia Barrett, Bobby Farrell and Liz Mitchell. The group's major break came in 1978 with ""Rivers Of Babylon"", the second highest selling single of all time in the UK. Boney M. enjoyed over a dozen hit singles throughout Europe between 1976 and 1981. After Farrell was replaced by Reggie Tsiboe, Boney M. had two more hits in 1984. Frank Farian announced the end of Boney M. in 1986, but the group has continued in some form or other ever since. No new material has been released, except for a couple of singles and the odd new track on compilations. Numerous ""best of"" compilations and remix albums have been released. All members continue performing with their own Boney M. formations even today, except for Tsiboe – and Farrell, who passed away in late 2010.",http://en.wikipedia.org/wiki/Boney_M.
59637,Anne Pigalle,None,None,http://www.annepigalle.com
48404,Tom Tom Club,Chris Frantz & Tina Weymouth,"U.S. group formed at the start of the '80's by Tina Weymouth and Chris Frantz of Talking Heads. Other members of the group were Steve Scales, Alex Weir, Tyrone Downie and Tina's sisters Loric, Lani and Laura.",http://tomtomclub.com
8024,Janet Jackson,Janet Damita Jo Jackson,"American recording artist and actress, born 16 May 1966 in Gary, Indiana, USA. Daughter of [a=Joe Jackson (5)], sister of [a=Michael Jackson], [a=Jermaine Jackson], [a=Jackie Jackson], [a=Tito Jackson], [a=Marlon Jackson], [a=Randy Jackson], [a=Rebbie Jackson] and [a=La Toya Jackson].",http://www.janetjackson.com


In [88]:
%%sql
WITH artists_by_num_releases AS (
    SELECT a.*, COUNT(DISTINCT r.release_id) AS num_releases
    FROM artists AS a
    JOIN released_by AS rb USING(artist_id)
    JOIN releases AS r USING(release_id)
    GROUP BY a.artist_id
),

artists_two_hundred AS (
    SELECT a.*
    FROM artists AS a
    JOIN artists_by_num_releases AS abnr USING(artist_id)
    WHERE abnr.num_releases > 200
),

artists_pop AS (
    SELECT a.*
    FROM artists AS a
    JOIN released_by AS rb USING(artist_id)
    JOIN releases AS r USING(release_id)
    WHERE r.genre = 'Pop'
)

SELECT ath.*
FROM artists_two_hundred AS ath
EXCEPT
SELECT ap.*
FROM artists_pop AS ap
LIMIT 10


 * postgresql://postgres:***@db:5432/postgres
10 rows affected.


artist_id,name,realname,profile,url
1279,Orbital,Paul & Phil Hartnoll,"Techno outfit, founded in the late 80s by the brothers Paul and Phil Hartnoll.",http://orbitalofficial.com
1280,Underworld,"Karl Hyde, Rick Smith, (Darren Emerson)","UK electronic music act formed by Rick Smith and [a=Karl Hyde] in 1987 after their former band [a=Freur] disbanded. The first line-up released more song oriented funky guitar based pop over the course of two albums. In 1991 the core duo returned with a new track oriented technoid direction and a new ally; DJ [a=Darren Emerson]. In 1996 the former b-side ""Born Slippy NUXX"" was used in Danny Boyle's film ""Trainspotting"" and became an instant anthem of the mid to late 1990's. Emerson left in 2000 after the release of ""Everything, Everything"" to pursue a solo career and was replaced on tour DJ duties by [a=Darren Price]. Price became a performing member of Underworld Live in 2005.",http://www.underworldlive.com
4118,Faithless,None,"Faithless officially split up in April 2011. Some of the original members (Sister Bliss, Maxi Jazz and percussionist Sudha Kheterpal) continue to perform under the name Faithless Sound System. Faithless were [a=Maxwell Frazer] AKA [a=Maxi Jazz] (Conscious Rap) [a=Ayalah Bentovim] AKA [a=Sister Bliss] (Music) Roland Armstrong AKA [a=Rollo] (Producer, Music). Many of the songs are sung by [a=Pauline Taylor]. [a=Jamie Catto] was originally part of the group but left after the release of the second album ""Sunday 8PM"". [a=Zoë Johnston] joined for the album ""Outrospective"". Their first album, 1996's ""Reverence"", was an underground sensation but did not quite break the mainstream. 1998's ""Sunday 8AM"" changed all of that, the single ""God Is a DJ"" became a big club anthem while the album enjoyed bigger sales than anything they had previously released. Critics may have been harsh, but the group kept moving forward with all members also pursuing solo careers on the side. When playing live they were joined by guitarist [a=Dave Randall (2)] and bassist, [a=Aubrey Nunn].",http://www.faithless.co.uk
5797,ATB,André Tanneberger,"German Producer ATB - [b]A[/b]ndré [b]T[/b]anne[b]b[/b]erger Born on February 26, 1973 in Freiberg, Saxony, Germany.",http://www.atb-music.com/
7142,The Art Of Noise,None,"The Art Of Noise, an art-pop group established in 1983, scored hits with ""Beat Box"", ""Close (to the Edit)"", ""Moments in Love"", ""Paranoimia"", and cover versions, with guest performers, of ""Peter Gunn"" and Prince's ""Kiss"". Often cited as a pioneer in electronic music and sampling, The Art Of Noise used technology to rework the same recordings many times over, a novel approach at the time. Although only a relatively small amount of original material was commercially released during the project's active periods (1983–1989 and 1998–2000), the Art Of Noise's legacy includes scores of reissues, remixes, and compilations of previously released and unreleased material. The Art Of Noise began as a project of studio engineer [a=Gary Langan] and Fairlight sampler whiz [a=J.J. Jeczalik], who were working together with producer/performer [a=Trevor Horn] and pianist/composer/arranger [a=Anne Dudley] on several of Horn's productions for other artists. Writer [a=Paul Morley] was brought in to tailor the group's image, which was at first that of a faceless ""non-group"" inspired by early 20th Century modernism. An acrimonious split in 1985 left Art Of Noise as a trio, sans Horn and Morley, and with a more tongue-in-cheek musical direction. Langan moved on amicably in 1987, leaving Dudley and Jeczalik to carry on until they, too, called it quits in 1990. In 1998, Horn, Dudley and Morley temporarily reformed The Art Of Noise with the addition of guitarist [a=Lol Creme]. The resulting album, [i]The Seduction of Claude Debussy[/i] (1999), was another attempt to raid the 20th Century's musical forms, this time focusing on the life and work of longtime influence Debussy. The group went dorm

## Exercise 4: Discuss query patterns and language features of SQL
1. What patterns did you use in many of the queries above? 

2. What is the usual pattern of an SQL query? Which operations happen pre-grouping and which ones post-grouping?

3. What makes SQL a declarative language and what advantages does that have?

4. What makes SQL a functional language and what advantages does that have?

5. How would the denormalization we talked about previously simplify the queries?

## Exercise 5: Limits of SQL (optional)
Explain what the following query does.
<b>Hints</b>: The query treats the data as if it was in graph shape.

In [55]:
%%sql
WITH RECURSIVE
    X AS (SELECT 3 AS Value),
    artist_releases AS (
        SELECT artists.artist_id, artists.name, releases.release_id, releases.title
        FROM artists, released_by, releases
        WHERE artists.artist_id = released_by.artist_id
        AND released_by.release_id = releases.release_id
    ),
    collaborations AS (
        SELECT DISTINCT ar1.artist_id AS left_id, ar1.name AS left_name, 
                ar2.artist_id AS right_id, ar2.name AS right_name, 1 AS distance
        FROM artist_releases AS ar1, artist_releases AS ar2
        WHERE ar1.release_id = ar2.release_id
        AND ar1.artist_id != ar2.artist_id
    ),
    X_hop_collaborations AS (
        SELECT * FROM collaborations  -- base case
        UNION
        SELECT c1.left_id, c1.left_name, c2.right_id, c2.right_name, c1.distance + 1 AS distance
        FROM X_hop_collaborations AS c1
        JOIN collaborations c2 ON c1.right_id = c2.left_id
        WHERE c1.distance < (SELECT * FROM X) AND c1.left_id != c2.right_id
    )
SELECT * 
FROM X_hop_collaborations
WHERE left_name = 'Coldplay'
ORDER BY distance, right_name;

 * postgresql://postgres:***@db:5432/postgres
23 rows affected.


left_id,left_name,right_id,right_name,distance
29735,Coldplay,1654,DK,1
29735,Coldplay,392179,G Synth,1
29735,Coldplay,10916,Jan Johnston,1
29735,Coldplay,1279,Orbital,1
29735,Coldplay,10785,Angelo Badalamenti,2
29735,Coldplay,11101,Cosmic Gate,2
29735,Coldplay,7090,Freefall,2
29735,Coldplay,2604010,Jada (7),2
29735,Coldplay,18836,Kirk Hammett,2
29735,Coldplay,11749,Knuckleheadz,2
